# Loading 3D objects from file
Requires Python>=3.7  

Here we showcase object creation by creating a network of filaments and motors.  

This requires compiling cytosim in 3 dimensions. For this edit the file "src/math/dim.h" to have "DIM = 3". Then compile cythosim and play, and move cytosim.---.so to the current folder.
```bash
$ make -j4 pycytosim play
$ cp bin/*.so ./examples/
```

In [1]:
import cytosim
import numpy as np
import pandas as pd

### Starting simulation
Here we start with fiber.cym for simplicity, and we erase the filament.

In [2]:
parser = cytosim.start("../cym/fiber.cym")
sim = parser.simul

In [3]:
sim.fibers.erase_all()
sim.once()

### Defining actin filament properties
We define actin as a semi-flexible filament, and define segmentation length ds as 3nm.

In [4]:
ds = 0.003
actin_props = "rigidity = 0.01 ; segmentation = %s ; confine = inside, 200, cell" %ds
parser.set("fiber","actin",actin_props);

### Loading from file 
Here we take file file_example_1_xCoeur749.384_yCoeur898.589.txt from the repository https://gitlab.com/SergeDmi/podosome-demo/-/tree/master/data

This contains a fraction of a podosome, in which actin filament have been segmented by electron tomography.

Download the file to the current folder to run this example. To download it, run : 
```bash
$ git clone git@gitlab.com:SergeDmi/podosome-demo.git pdemo
$ cp pdemo/data/file_example_1_xCoeur749.384_yCoeur898.589.txt .
```



In [5]:
filename = "file_example_1_xCoeur749.384_yCoeur898.589.txt"
raw_fibs = pd.read_csv(filename, header=None,).to_numpy()

In [6]:
# column 4 contain the filament number
# column 1-3 contain the x,y,z coordinates
for ix in np.unique(raw_fibs[:,4]):
    pts = raw_fibs[raw_fibs[:,4]==ix,1:4]/1000.0
    length = (pts.shape[0] - 1)*ds
    # adding one filament with the adequate number of points
    newfil = sim.add("actin", "length=%s"%length, 1)[0]
    # replacing these points by those read in the files
    np.array(newfil.data(),copy=False)[:,:] = pts

In [7]:
# Running several timesteps and saving
parser.save()
for i in range(10):
    sim.once()
    parser.save()

### Changing segmentation size
We can change the segmentation of the imported filaments to allow for faster simulations.

In [8]:
parser.change("actin","segmentation = %s" %(ds*5))

In [9]:
parser.save()
for i in range(20):
    sim.once()
    parser.save()

### Playing the results
This requires play to be compiled in the folder /bin.  

In [ ]:
# Playing ()
!./bin/play

chain f2:0369
{
    segmentation = 0.003
    segment_min = 0.00280314
    segment_max = 0.00299999
    length  = 0.228
    contour = 0.225235
}
chain f2:0322
{
    segmentation = 0.003
    segment_min = 0.00283317
    segment_max = 0.0029989
    length  = 0.102
    contour = 0.101254
}
chain f2:0297
{
    segmentation = 0.003
    segment_min = 0.00284855
    segment_max = 0.00299997
    length  = 0.072
    contour = 0.070953
}
chain f2:0089
{
    segmentation = 0.003
    segment_min = 0.00280177
    segment_max = 0.00300001
    length  = 0.348
    contour = 0.345389
}
